In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns',None)

In [8]:
defenses = pd.read_csv("../processed_data/processing_defenses/defense_players_plays_no_rz.csv",index_col=0)
defenses.head()

,id,x,y,s,a,dis,o,dir,nflId,displayName,jerseyNumber,position,team,playDirection
1,2018090600:75:1,88.89,36.47,0.01,0.01,0.01,105.63,66.66,79848.0,Malcolm Jenkins,27.0,SS,home,left
3,2018090600:75:1,86.31,22.01,0.09,0.42,0.01,70.12,168.91,2495613.0,Corey Graham,24.0,FS,home,left
5,2018090600:75:1,73.64,28.70,0.03,0.02,0.01,103.05,219.41,2534832.0,Rodney McLeod,23.0,FS,home,left
7,2018090600:75:1,86.48,31.12,0.09,0.03,0.04,95.90,33.36,2552315.0,Jordan Hicks,58.0,MLB,home,left
8,2018090600:75:1,82.67,20.53,0.03,0.07,0.01,81.14,174.57,2552689.0,Ronald Darby,21.0,CB,home,left


In [5]:
plays = pd.read_csv("../processed_data/processing_defenses/plays.csv",index_col=0)
plays.head()

,id,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI,redZone
0,2018090600:75:1,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False,False
1,2018090600:146:1,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False,False
2,2018090600:168:1,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False,False
3,2018090600:190:1,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False,False
4,2018090600:256:1,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,SHOTGUN,"2 RB, 3 TE, 0 WR",8.0,6.0,"6 DL, 3 LB, 2 DB",0.0,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False,True


In [7]:
plays_no_rz = plays[plays["redZone"]==False]
plays_no_rz.head()

,id,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI,redZone
0,2018090600:75:1,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False,False
1,2018090600:146:1,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False,False
2,2018090600:168:1,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False,False
3,2018090600:190:1,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False,False
5,2018090600:320:1,(10:10) (Shotgun) N.Foles pass short left to N...,1,2,8,PHI,play_type_pass,PHI,4,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,4.0,"3 DL, 3 LB, 5 DB",0.0,0.0,10:10:00,14.0,NaN,NaN,C,4,4,-0.344096,False,False


Vamos a empezar identificando cuantos jugadores hay en la zona profunda. Esta zona está 10 o 15 yardas más lejos de la línea de scrimmage.